In [344]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime, date, timedelta
from time import sleep
import numpy as np
import pandas as pd
import re

In [375]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
# driver.get("https://www.groupon.com/deals/n-bjs-wholesale-club-memberships") #BJ's
driver.get("https://www.groupon.com/deals/n-sams-club-membership-packages-7#tips") #Sam's Club
driver.maximize_window()

In [377]:
all_reviews = driver.find_element_by_xpath("/html/body/section[2]/div/section/div/div[2]/div[2]/section/div/div/article/div[3]/div[2]/div[2]/a")

all_reviews.click()

In [128]:
def scrape():
    temp = []
    time = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
    contents =  driver.find_elements_by_class_name("tip-item")
    for content in contents:
        temp.append(content.text + f"\n{time}")
    return temp[8:]

In [379]:
temp = []
for i in range(2828):
    for j in scrape():
        temp.append(j)
    next_button = driver.find_element_by_xpath("/html/body/div[*]/div[2]/div[5]/div[2]")
    next_button.click()
    sleep(3)
reviews = pd.DataFrame(temp, columns= ["full"])

In [232]:
# reviews["split"] =reviews["full"].str.split("\n")
# reviews["length"] = [len(i) for i in reviews.split]

In [269]:
# # Not all reviews are the same. Some missing elements
# reviews.length.value_counts()
# reviews.loc[reviews.length == 5,"split"][4141]
# reviews.loc[reviews.length == 6,"split"][600]
# reviews.loc[reviews.length == 7,"split"][0]

In [ ]:
# reviews["title"] = ""
# reviews["subtitle"] = ""
# reviews["date_published"] = ""
# reviews["text"] = ""
# reviews["date_scraped"] = ""

In [358]:
# for i, split in enumerate(reviews.split):
#     if len(split) == 7:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = split[4]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[6]

    
#     elif len(split) == 6:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = "None"
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]

#     elif len(split) == 5:
#         reviews.loc[reviews.index == i,"title"] = "None"
#         reviews.loc[reviews.index == i,"subtitle"] = split[0]
#         reviews.loc[reviews.index == i,"date_published"] = split[1]
#         reviews.loc[reviews.index == i,"text"] = split[2]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[4]

In [271]:
# # Checked reviews 
# reviews.loc[reviews.length == 5,]
# reviews.loc[reviews.length == 6,]
# reviews.loc[reviews.length == 7,]

,full,split,length,title,subtitle,date_published,date_scraped,text
4141,"2 ratings2 reviews\nMay 12, 2016\nWorth it\nHe...","[2 ratings2 reviews, May 12, 2016, Worth it, H...",5,None,2 ratings2 reviews,"May 12, 2016",03/27/2021 21:50:51,Worth it
4677,"6 ratings3 reviews\nMay 12, 2016\nBj's has rea...","[6 ratings3 reviews, May 12, 2016, Bj's has re...",5,None,6 ratings3 reviews,"May 12, 2016",03/27/2021 21:54:10,Bj's has really improved. Great selection of g...


In [310]:
# # Found an anomly with review 4661 because they are a top reviewer with no name
# reviews.split[4661]
# reviews.loc[reviews.index == 4661,"title"] = "None"+reviews.split[4661][0]
# reviews.loc[reviews.index == 4661,"subtitle"] = reviews.split[4661][1]
# reviews.loc[reviews.index == 4661,"date_published"] = reviews.split[4661][2]
# reviews.loc[reviews.index == 4661,"text"] = reviews.split[4661][3]
# reviews.loc[reviews.index == 4661,"date_scraped"] = reviews.split[4661][5]
# reviews.loc[reviews.index == 4661,]


,full,split,length,title,subtitle,date_published,date_scraped,text,top_reviewer,helpful_reviewer,rating,review_count,name
4661,TOP REVIEWER\n15 ratings14 reviews\nFebruary 1...,"[TOP REVIEWER, 15 ratings14 reviews, February ...",6,NoneTOP REVIEWER,15 ratings14 reviews,"February 15, 2018",03/27/2021 21:54:07,Still awaiting answer on if membersship is for...,1,0,15,14,None


In [309]:
reviews["top_reviewer"] = [1 if ("TOP REVIEWER" in i) else 0 for i in reviews.title]
reviews["helpful_reviewer"] = [1 if ("HELPFUL REVIEWER" in i) else 0 for i in reviews.title]
reviews["rating"] = [int(re.findall(r'\d+',i)[0]) for i in reviews.subtitle]
reviews["review_count"] = [int(re.findall(r'\d+',i)[1]) for i in reviews.subtitle]
reviews["name"] = [re.split('TOP REVIEWER|HELPFUL REVIEWER',i)[0] for i in reviews.title]

In [362]:
# for i, date in enumerate(reviews.date_published[:6]):

#     if "hour" in date:
#         reviews.loc[reviews.index == i,"date_published"] = reviews.date_scraped[i][:10]
#     elif "days" in date:
#         scraped = datetime.strptime(reviews.date_scraped[i],"%m/%d/%Y %H:%M:%S").date()
#         ago = timedelta(days = int(re.findall(r'\d+',date)[0]))
#         reviews.loc[reviews.index == i,"date_published"] = (scraped - ago).strftime("%m/%d/%Y")

    
    

In [371]:
# reviews.to_csv('Groupon_BJs_Raw.csv', index=False)
# reviews[['name', 'text', 'rating', 'review_count','top_reviewer', 'helpful_reviewer','date_published','date_scraped']].to_csv('Groupon_BJs.csv', index=False)